In [43]:
import pandas as pd # To manage data as data frames
import numpy as np # To manipulate data as arrays
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

import pickle

In [44]:
# import regression module
#from sklearn.linear_model import LinearRegression

# split predictors and response
#X = df.drop(['mpg'], axis=1)
#y = df['mpg']

In [45]:
df_train = pd.read_csv("df_train.csv")
df_train.head()

,Unnamed: 0,time,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_clouds_all,Bilbao_wind_speed,...,Madrid_temp_max,Barcelona_temp,Bilbao_temp_min,Bilbao_temp,Barcelona_temp_min,Bilbao_temp_max,Seville_temp_min,Madrid_temp,Madrid_temp_min,load_shortfall_3h
0,0,2015-01-01 03:00:00,0.666667,level_5,0.0,0.666667,74.333333,64.000000,0.0,1.000000,...,265.938000,281.013000,269.338615,269.338615,281.013000,269.338615,274.254667,265.938000,265.938000,6715.666667
1,1,2015-01-01 06:00:00,0.333333,level_10,0.0,1.666667,78.333333,64.666667,0.0,1.000000,...,266.386667,280.561667,270.376000,270.376000,280.561667,270.376000,274.945000,266.386667,266.386667,4171.666667
2,2,2015-01-01 09:00:00,1.000000,level_9,0.0,1.000000,71.333333,64.333333,0.0,1.000000,...,272.708667,281.583667,275.027229,275.027229,281.583667,275.027229,278.792000,272.708667,272.708667,4274.666667
3,3,2015-01-01 12:00:00,1.000000,level_8,0.0,1.000000,65.333333,56.333333,0.0,1.000000,...,281.895219,283.434104,281.135063,281.135063,283.434104,281.135063,285.394000,281.895219,281.895219,5075.666667
4,4,2015-01-01 15:00:00,1.000000,level_7,0.0,1.000000,59.000000,57.000000,2.0,0.333333,...,280.678437,284.213167,282.252063,282.252063,284.213167,282.252063,285.513719,280.678437,280.678437,6620.666667


In [46]:
df_train_clean = df_train.copy()

In [47]:
df_train_clean['time'] = pd.to_datetime(df_train_clean['time'])

In [48]:
df_train_clean['Valencia_wind_deg'] = df_train_clean['Valencia_wind_deg'].str.extract('(\d+)')

In [49]:
df_train_clean['Valencia_wind_deg'] = pd.to_numeric(df_train_clean['Valencia_wind_deg'])

In [50]:
df_train_clean['Seville_pressure'] = df_train_clean['Seville_pressure'].str.extract('(\d+)')

In [51]:
df_train_clean['Seville_pressure'] = pd.to_numeric(df_train_clean['Seville_pressure'])

In [52]:
# Replace NaN with Mean for Valencia Pressure
mean_Valencia_pressure=df_train_clean.Valencia_pressure.mean()
df_train_clean.loc[df_train_clean.Valencia_pressure.isnull(),'Valencia_pressure']=mean_Valencia_pressure

In [53]:
df_train_clean = df_train_clean.drop(['Unnamed: 0'], axis = 1)

In [54]:
df_train_clean_no_time = df_train_clean.copy()

In [55]:
df_train_clean_no_time = df_train_clean_no_time.drop(['time'], axis = 1)

In [56]:
corr_matrix = df_train_clean_no_time.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
print(to_drop)

['Valencia_temp', 'Seville_temp', 'Valencia_temp_min', 'Barcelona_temp_max', 'Madrid_temp_max', 'Barcelona_temp', 'Bilbao_temp', 'Barcelona_temp_min', 'Bilbao_temp_max', 'Seville_temp_min', 'Madrid_temp', 'Madrid_temp_min']


In [57]:
df_train_clean_no_time.drop(to_drop, axis=1, inplace=True)

In [58]:
y = df_train_clean_no_time.load_shortfall_3h
X = df_train_clean_no_time.drop(['load_shortfall_3h'], axis=1)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)

In [60]:
# create model object
randomfor =RandomForestRegressor()
randomfor = randomfor.fit(train_X,train_y)

In [63]:
y_pred = randomfor.predict(X)

In [64]:
model_save_path = "lin_model.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(lin_model, file)

In [65]:
model_save_path = "randomfor_model.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(randomfor, file)